<a href="https://colab.research.google.com/github/revs1/Natural-Language-Processing/blob/master/Detect_emotions_Spark_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd /content
!sudo apt-get install openjdk-8-jdk
!java -version
!pip install --ignore-installed -q pyspark==2.4.4
!pip install spark-nlp

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk is already the newest version (8u252-b09-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
openjdk version "11.0.7" 2020-04-14
OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode)


In [3]:
import pandas as pd
import numpy as np
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

In [4]:
spark = sparknlp.start()

In [5]:
MODEL_NAME='classifierdl_use_emotion'

In [6]:

text_list = [
            """I am SO happy the news came out in time for my birthday this weekend! My inner 7-year-old cannot WAIT!""",
            """That moment when you see your friend in a commercial. Hahahaha!""",
            """My soul has just been pierced by the most evil look from @rickosborneorg. A mini panic attack &amp; chill in bones followed soon after.""",
            """For some reason I woke up thinkin it was Friday then I got to school and realized its really Monday -_-""",
            """I'd probably explode into a jillion pieces from the inablility to contain all of my if I had a Whataburger patty melt right now. #drool""",
            """These are not emotions. They are simply irrational thoughts feeding off of an emotion""",
            """Found out im gonna be with sarah bo barah in ny for one day!!! Eggcitement :)""",
            """That awkward moment when you find a perfume box full of sensors!""",
            """Just home from group celebration - dinner at Trattoria Gianni, then Hershey Felder's performance - AMAZING!!""",
            """Nooooo! My dad turned off the internet so I can't listen to band music!""",
            ]

In [7]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = ClassifierDLModel.pretrained(name=MODEL_NAME)\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_emotion download started this may take some time.
Approximate size to download 20.7 MB
[OK!]


In [8]:

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = pipelineModel.transform(df)

In [9]:
result.select(F.explode(F.arrays_zip('document.result', 'sentiment.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("sentiment")).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------+---------+
|document                                                                                                                               |sentiment|
+---------------------------------------------------------------------------------------------------------------------------------------+---------+
|I am SO happy the news came out in time for my birthday this weekend! My inner 7-year-old cannot WAIT!                                 |surprise |
|That moment when you see your friend in a commercial. Hahahaha!                                                                        |surprise |
|My soul has just been pierced by the most evil look from @rickosborneorg. A mini panic attack &amp; chill in bones followed soon after.|fear     |
|For some reason I woke up thinkin it was Friday then I got to school and realized its really Monday -_-        